# Recipe 평가 노트북
레시피 생성 결과를 LLM을 통해 평가하기 위한 프롬프트와 함수를 정의합니다.

In [10]:
import os
from typing import Dict
import openai
from model.recipe_model import recipe_llm
from utils.prompt_loader import load_prompt
# OpenAI API 키 설정 (환경 변수 OPENAI_API_KEY 사용)
recipe_llm = recipe_llm

In [ ]:
# 평가 프롬프트 템플릿 정의
qa_evaluate_template = load_prompt("constitution_recipe_evaluate/constitution_recipe_question.json")
recipe_evaluate_template = load_prompt("constitution_recipe_evaluate/constitution_recipe_question.json")

In [13]:
history = """user: 안녕
assistant: 안녕하세요! 체질 기반 건강 요리 레시피를 도와드리기 위해 몇 가지 질문을 할게요. 먼저, 고객님의 체질은 무엇인가요? (목양체질, 목음체질, 토양체질, 토음체질, 금양체질, 금음체질, 수양체질, 수음체질 중 선택해주세요.)
user: aa
assistant: 체질 선택이 'aa'로 나타났는데요, 혹시 다시 정확한 체질을 알려주실 수 있나요? 또는 알레르기, 건강 상태, 식이 제한, 원하는 음식 종류, 조리 도구, 인원수도 함께 알려주시면 도움이 될 것 같아요.
user: 목양체질이야
assistant: 좋은 정보 감사합니다! 다음으로 고객님의 알레르기 또는 특별히 피하고 싶은 음식이 있나요? (예: 계란, 견과류, 밀가루 등)
user: 그런거 없어
assistant: 건강 상태나 특별히 제한하는 식이 요법이 있나요? (예: 다이어트, 저염식, 당뇨 관련 식단 등)
user: 네
assistant: 혹시 그 제한하는 식이 요법이 어떤 것인지 구체적으로 말씀해 주시겠어요?
user: 없어
assistant: 좋습니다! 그럼 이제 고객님이 선호하시는 음식 종류 또는 드시고 싶은 음식이 있나요? (예: 한식, 중식, 일식, 양식, 채식 등)
user: 아니 없어
assistant: 조리 도구나 특별히 사용하고 싶은 조리 방법이 있나요? (예: 냄비, 프라이팬, 오븐, 찜기 등)
user: 없어
assistant: 몇 인분이 필요한지 알려주실 수 있나요? (예: 1인분, 2인분, 4인분 등)
user: 2인분 요리 만들어줘
assistant: [{"id": "6812d19b348234a7ffae0361", "title": "목양체질을 위한 건강한 한식 요리", "description": "목양체질에 맞게 기를 보하고 스트레스를 완화하는 건강한 한식 요리입니다. 신선한 채소와 영양 가득한 재료를 활용하여 맛과 건강을 동시에 챙기세요.", "difficulty": "중급", "cookTime": "30분", "ingredients": ["두부 1모", "시금치 200g", "당근 1개", "파 2대", "마늘 3쪽", "들기름 1큰술", "간장 2큰술", "소금 약간", "참기름 1작은술", "통깨 약간"], "image": "https://example.com/image.jpg", "rating": 4.5, "suitableFor": "목양체질을 가진 사람들, 건강식이 필요한 분들", "suitableBodyTypes": ["목양체질"], "reason": "목양체질에 맞게 혈액 순환과 기력 회복을 돕는 재료들을 활용한 레시피입니다.", "tags": ["한식", "건강식", "목양체질", "저염식"], "steps": ["시금치는 깨끗이 세척한 후 데쳐서 찬물에 헹구고, 먹기 좋은 크기로 썬다.", "당근은 채 썰고, 파는 송송 썬다.", "마늘은 다지고, 두부는 먹기 좋은 크기로 자른 후 기름을 두른 팬에 살짝 구워둔다.", "팬에 들기름을 넣고 다진 마늘과 당근을 먼저 볶는다.", "당근이 익으면 데친 시금치를 넣고 함께 볶는다.", "간장과 소금을 넣어 간을 맞춘 후, 두부를 넣고 살살 섞는다.", "마지막으로 참기름과 통깨를 뿌려 완성한다."], "servings": "2인분", "nutritionalInfo": "이 레시피는 혈액 순환을 도우며, 목양체질에 적합한 기력 회복에 도움을 주는 영양가 높은 식단입니다."}]"""

In [16]:
def evaluate_qa_recipe(history: str) -> Dict:
    '''
    주어진 레시피를 LLM으로 평가하고 JSON으로 결과를 반환합니다.
    '''
    prompt = qa_evaluate_template.format(qa_list=history)
    response = recipe_llm.invoke([{'role': 'user', 'content': prompt}])
    
  
    content = response.content
    print(content)
    # JSON 파싱
    try:
        import json
        result = json.loads(content)

    except Exception as e:
        raise ValueError(f'JSON 파싱 실패: {e} 응답 내용: {content}')
    return result

In [17]:
# 예제 실행
if __name__ == '__main__':
    evaluation = evaluate_recipe(history)
    print(evaluation)

[
  {
    "question": "'user'의 질병 상황(있음/없음 여부 포함)이 명시적으로 언급되었는가?",
    "answer": "아니요",
    "reason": "'user'는 건강 상태 또는 질병 유무에 대해 언급하지 않았으며, 단순히 '특별히 제한하는 식이 요법이 있나요?'에 대해 '없어'라고 답변했을 뿐 질병 관련 내용을 구체적으로 말하지 않음."
  },
  {
    "question": "'user'의 체질 유형(예: 태양인, 소양인 등)이 명확하게 표현되었는가?",
    "answer": "예",
    "reason": "'user'는 '목양체질이야'라고 명확하게 체질을 언급함."
  },
  {
    "question": "'user'의 특정 알레르기 유무(있음/없음)가 명시적으로 언급되었는가?",
    "answer": "예",
    "reason": "'user'는 알레르기 또는 피하고 싶은 음식에 대해 '그런거 없어'라고 답변하여 명확하게 언급함."
  },
  {
    "question": "'user'의 특정 식재료 또는 조리법에 대한 선호/비선호 정보의 유무가 명시적으로 언급되었는가?",
    "answer": "아니요",
    "reason": "'user'는 조리법이나 특정 재료에 대한 선호나 비선호를 묻는 질문에 대해 언급하지 않았으며, 단순히 '없어'라고 답변했음."
  },
  {
    "question": "'user'가 선호하거나 피하고자 하는 음식 종류(예: 한식, 일식, 채식 등)를 언급하였는가?",
    "answer": "예",
    "reason": "'user'는 '아니 없어'라고 답하며 별도의 음식 종류 선호를 언급하지는 않았으나, 질문 자체에서 '선호하는 음식 종류'를 묻는 것에 답변하였기 때문에 '아니 없어'로 언급된 내용이 명확히 체크됨."
  },
  {
    "question": "'user'가 사용할 수 있는 조리도구에 대한 정보를 제공하였는가?",
    

In [19]:
def evaluate_metric(evaluate_json):
    count = 0
    for qa in evaluate_json:
        if qa['answer'] == '예':
            count += 1
    return count / len(evaluate_json)
    


answer:  아니요
answer:  예
answer:  예
answer:  아니요
answer:  예
answer:  아니요
answer:  예
answer:  아니요


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from core.config import settings
 
# Initialize model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=settings.GEMINI_API_KEY)
 
result = llm.invoke("Explain the concept of prompt engineering in one sentence.")
 
print(result.content)
print("\nUsage Metadata:")
print(result.usage_metadata)

Prompt engineering is the art of crafting effective text-based prompts to guide large language models to generate desired and accurate outputs.

Usage Metadata:
{'input_tokens': 10, 'output_tokens': 25, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}}


In [6]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI
from core.config import settings
llm = ChatOpenAI(model_name="gpt-4.1-nano-2025-04-14",api_key=settings.OPENAI_API_KEY)

with get_openai_callback() as cb:
    result = llm.invoke("대한민국의 수도는 어디야?")
    print(cb)

Tokens Used: 24
	Prompt Tokens: 15
		Prompt Tokens Cached: 0
	Completion Tokens: 9
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $5.1e-06


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback
from core.config import settings
llm = ChatOpenAI(model="gpt-4",api_key=settings.OPENAI_API_KEY)

response = llm.invoke("Tell me a joke.")
response
input_tokens = response.response_metadata["token_usage"]["prompt_tokens"]
output_tokens = response.response_metadata["token_usage"]["completion_tokens"]


AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 12, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BU3sSyEmBxZwdoFaib11HF1Sy8Qaa', 'finish_reason': 'stop', 'logprobs': None}, id='run--dc8db3ab-2d43-4a2e-9c58-aecaaa5257a5-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
from langchain_anthropic import ChatAnthropic
from core.config import settings
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219",api_key=settings.CLAUDE_API_KEY)
response = llm.invoke("Tell me a joke.")
input_tokens = response.response_metadata["usage"]["input_tokens"]
output_tokens = response.response_metadata["usage"]["output_tokens"]

AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, response_metadata={'id': 'msg_01LH6TFEvyzxDTouwWVpkjnf', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 17}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--85f8063d-aa7a-455e-a65b-d3e502102aef-0', usage_metadata={'input_tokens': 12, 'output_tokens': 17, 'total_tokens': 29, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI
from core.config import settings
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key=settings.GEMINI_API_KEY)
response = llm.invoke("Tell me a joke.")
response
input_tokens = response.usage_metadata["input_tokens"]
output_tokens = response.usage_metadata["output_tokens"]




17

In [29]:
response.usage_metadata["input_tokens"]
response.usage_metadata["output_tokens"]

17